In [1]:
# Dependencies
from citipy import citipy as ct
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy.stats as sts
import gmaps
from config import weather_key, gkey

gmaps.configure(api_key=gkey)

In [2]:
# Get list of all unique cities available through citipy
coords = list(ct.WORLD_CITIES_DICT.keys())
city_list = list(set([ct.nearest_city(coords[index][0],coords[index][1]).city_name for index in range(len(coords))]))

In [3]:
# create weather dataframe
weather_data = pd.DataFrame(columns=['city','country','lat/lng','temp','hum','cld','wnd'])

# loop through a random sample of 650 cities and store weather information in dataframe
# some cities not available in weather api, so sample of 650 is used to compansate and get at least 500 in the dataframe
for city in random.sample(city_list,650):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=imperial&appid={weather_key}"
    response = requests.get(url).json()
    try:
        weather_data = weather_data.append({'city': city,
                                            'country': response['sys']['country'],
                                            'lat/lng': [response['coord']['lat'], response['coord']['lon']],
                                            'temp': response['main']['temp'],
                                            'hum': response['main']['humidity'],
                                            'cld': response['clouds']['all'],
                                            'wnd': response['wind']['speed']},ignore_index=True)
    except KeyError:
        # annotate which cities were skipped over
        print(f"No weather information found for city: {city}")

# confirm number of cities in final dataframe
print(f"Samples pulled: {len(weather_data)}")

# clean datatype for the two columns
weather_data[['hum','cld']] = weather_data[['hum','cld']].astype(float)

weather_data.head()

No weather information found for city: san celestino
No weather information found for city: tanjong sepat
No weather information found for city: tempoal
No weather information found for city: san ignacio de sabaneta
No weather information found for city: zmievka
No weather information found for city: evenskjaer
No weather information found for city: dalneye konstantinovo
No weather information found for city: bjala
No weather information found for city: kastellion
No weather information found for city: letogo
No weather information found for city: barghis
No weather information found for city: mau aima
No weather information found for city: waksma
No weather information found for city: ixcatlan
No weather information found for city: camp de masque pave
No weather information found for city: valea sarii
No weather information found for city: sperkhias
No weather information found for city: bolshiye kolpany
No weather information found for city: muhlheim
No weather information found for 

,city,country,lat/lng,temp,hum,cld,wnd
0,gotemba,JP,"[35.3, 138.9333]",64.11,22.0,1.0,18.41
1,bindoy,PH,"[9.7558, 123.1408]",87.80,66.0,40.0,9.22
2,solsona,ES,"[41.994, 1.5171]",49.30,83.0,100.0,1.01
3,timmins,CA,"[48.4669, -81.3331]",12.20,92.0,1.0,3.60
4,garagoa,CO,"[5.0824, -73.3633]",60.73,94.0,97.0,1.90


In [4]:
vacation = weather_data.loc[(weather_data.cld==0)&((weather_data.temp<80) & (weather_data.temp>65))&(weather_data.wnd<10)]
vacation = vacation.reset_index(drop=True)
vacation

,city,country,lat/lng,temp,hum,cld,wnd
0,aquiraz,BR,"[-3.9014, -38.3911]",76.37,94.0,0.0,5.75
1,jaragua,BR,"[-15.7569, -49.3344]",66.20,82.0,0.0,5.75
2,manjacaze,MZ,"[-24.7117, 33.8828]",73.09,78.0,0.0,3.65
3,batote,IN,"[33.1227, 75.321]",73.90,29.0,0.0,3.02
4,rolandia,BR,"[-23.3097, -51.3692]",66.20,82.0,0.0,4.61
5,santo anastacio,BR,"[-21.8383, -51.6103]",66.20,77.0,0.0,5.91
6,flinders,AU,"[-34.5833, 150.8552]",66.99,69.0,0.0,3.00
7,rio das pedras,BR,"[-22.8433, -47.6061]",67.71,82.0,0.0,5.75
8,sainte-rose,GP,"[16.3324, -61.6979]",73.40,78.0,0.0,4.61
9,sao roque,BR,"[-23.5292, -47.1353]",65.34,82.0,0.0,5.75


In [5]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = pd.DataFrame(columns=['Hotel Name','City','Country'])
for index,row in vacation.iterrows():
    params = {"location": str(row['lat/lng'])[1:-1],
              "radius": 5000,
              "type": 'hotel',
              "key": gkey}

    response = requests.get(url, params=params).json()
    name = response['results'][0]['name']
    hotels = hotels.append({'Hotel Name': name,
                   'City': row.city,
                   'Country': row.country},ignore_index=True)

hotels

,Hotel Name,City,Country
0,Eusébio,aquiraz,BR
1,Jaraguá,jaragua,BR
2,Manjacaze,manjacaze,MZ
3,Batote,batote,IN
4,Rolândia,rolandia,BR
5,Ribeirão dos Índios,santo anastacio,BR
6,Wollongong,flinders,AU
7,Jardim Dona Antonieta,rio das pedras,BR
8,Sainte-Rose,sainte-rose,GP
9,São Roque,sao roque,BR


In [6]:
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(weather_data['lat/lng'],
                              weights=weather_data.hum,
                              max_intensity=200,
                              dissipating=False,
                              point_radius=3)
# info box code based off tutorial: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</d><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotels.to_dict(orient='records')]

marker_layer = gmaps.marker_layer(vacation['lat/lng'],info_box_content=hotel_info)
fig.add_layer(heatmap)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))